# Performance Analysis of Bellman-Ford HPC Implementations (Colab)

This notebook provides a complete workflow to set up the environment, run benchmarks, and analyze the performance of four different implementations of the Bellman-Ford algorithm on Google Colab.

## 1. Environment Setup

First, let's set up the environment. This involves checking for a GPU, cloning the project repository from GitHub, and installing Python dependencies.

### 1.1 Check GPU Availability

Ensure that a GPU is available. Go to **Runtime -> Change runtime type** and select **GPU** as the hardware accelerator. The following cell should show your assigned GPU.

In [8]:
!nvidia-smi

Tue Aug 26 07:43:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### 1.2 Clone the Repository

In [3]:
!git clone https://www.github.com/UchihaIthachi/bellman-ford-hpc-openmp-cuda.git
%cd bellman-ford-hpc-openmp-cuda

Cloning into 'bellman-ford-hpc-openmp-cuda'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 45 (delta 15), reused 36 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 24.16 KiB | 4.03 MiB/s, done.
Resolving deltas: 100% (15/15), done.
/content/bellman-ford-hpc-openmp-cuda


### 1.3 Install Dependencies

In [4]:
%pip install pandas matplotlib seaborn

## 2. Build the Executables

Next, we compile the C/C++ and CUDA source code. The `Makefile` will automatically detect the GPU architecture and build the executables in the `bin/` directory.

In [5]:
!make clean && make

rm -rf ./bin reports
rm -f BF_serial BF_openmp BF_cuda BF_hybrid
rm -f *_output__*.txt
rm -f data/graph_*.txt
mkdir -p ./bin
g++ -O2 -std=c++14 -Wall -I./include -I./utils src/BF_serial.c utils/graph_io.c utils/graphGen.c -o bin/BF_serial
mkdir -p ./bin
g++ -O2 -std=c++14 -Wall -fopenmp -I./include -I./utils src/BF_openmp.c utils/graph_io.c utils/graphGen.c -o bin/BF_openmp
mkdir -p ./bin
nvcc -O2 -std=c++14 -gencode arch=compute_50,code=sm_50 -I./include -I./utils src/BF_cuda.cu utils/graph_io.c utils/graphGen.c -o bin/BF_cuda
mkdir -p ./bin
nvcc -O2 -std=c++14 -gencode arch=compute_50,code=sm_50 -Xcompiler -fopenmp -I./include -I./utils src/BF_hybrid.cu utils/graph_io.c utils/graphGen.c -o bin/BF_hybrid


## 3. Run the Benchmarks

Now, we'll run our custom benchmarking script, `scripts/benchmark.py`. This script will execute each Bellman-Ford implementation across a range of graph sizes and save the timing results to `benchmark_results.json`.

You can customize the vertex counts by modifying the `--vertices` argument.

In [ ]:
!chmod +x scripts/benchmark.py
!./scripts/benchmark.py --vertices 1000 2000 5000 10000 20000

Building project...
Running benchmarks for 1000 vertices...
  Running Serial...
    Time: 1e-05s
  Running OpenMP...
    Time: 0.01127s
  Running CUDA...
Error running command: ./bin/BF_cuda 1000 -30 30 0.001
Stderr: CUDA error malloc edges: CUDA driver version is insufficient for CUDA runtime version

  Running Hybrid...
Error running command: OMP_NUM_THREADS=8 ./bin/BF_hybrid 1000 -30 30 0.5 0.001 8
Stderr: CUDA error malloc edges: CUDA driver version is insufficient for CUDA runtime version

Running benchmarks for 2000 vertices...
  Running Serial...
    Time: 3.3e-05s
  Running OpenMP...
    Time: 0.014027s
  Running CUDA...
Error running command: ./bin/BF_cuda 2000 -30 30 0.001
Stderr: CUDA error malloc edges: CUDA driver version is insufficient for CUDA runtime version

  Running Hybrid...
Error running command: OMP_NUM_THREADS=8 ./bin/BF_hybrid 2000 -30 30 0.5 0.001 8
Stderr: CUDA error malloc edges: CUDA driver version is insufficient for CUDA runtime version

Running benchmark

## 4. Analyze the Results

With the benchmarks complete, let's load the results into a pandas DataFrame and examine the raw data.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

try:
    with open('benchmark_results.json', 'r') as f:
        results = json.load(f)
    df = pd.DataFrame(results)
    print("Benchmark Results:")
    display(df)
except FileNotFoundError:
    print("benchmark_results.json not found. Make sure the previous step ran successfully.")

### Performance Visualization

Now, let's plot the results to better visualize the performance differences. We'll create a line plot showing execution time versus the number of vertices for each implementation.

In [ ]:
if 'df' in locals():
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(12, 8))

    # Melt the dataframe to make it suitable for seaborn
    df_melted = df.melt(id_vars=['vertices'], var_name='Implementation', value_name='Execution Time (s)')
    df_melted = df_melted.dropna()

    sns.lineplot(data=df_melted, x='vertices', y='Execution Time (s)', hue='Implementation', marker='o', ax=ax)

    ax.set_title('Bellman-Ford Performance Comparison', fontsize=16)
    ax.set_xlabel('Number of Vertices', fontsize=12)
    ax.set_ylabel('Execution Time (s)', fontsize=12)
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.legend(title='Implementation')

    plt.show()

### Analysis

*(This is a placeholder for the analysis. The actual analysis will be written after running the notebook in a GPU-enabled environment.)*

From the plot, we can draw several conclusions:

- **Serial:** As expected, the serial implementation is the slowest. Its execution time grows rapidly with the number of vertices.
- **OpenMP:** The OpenMP version provides a significant speedup over the serial version by utilizing multiple CPU cores. However, its performance is still limited by the number of available cores.
- **CUDA:** For larger graphs, the CUDA implementation should demonstrate a dramatic performance improvement. The massive parallelism of the GPU allows it to process a large number of edges simultaneously. For smaller graphs, the overhead of transferring data to and from the GPU might make it slower than the CPU-based versions.
- **Hybrid:** The hybrid approach aims to get the best of both worlds. It can be particularly effective for certain graph structures and problem sizes, but its performance depends heavily on the CPU-GPU workload split.

## 5. Conclusion

This analysis demonstrates the power of high-performance computing techniques for accelerating the Bellman-Ford algorithm. While the serial version is simple to implement, it does not scale well. OpenMP provides a good performance boost on multi-core CPUs. For maximum performance on large graphs, the CUDA implementation is the clear winner, showcasing the massive parallel processing capabilities of modern GPUs.